Boutique Management System using Python-MySQL Connectivity

In [ ]:
import tkinter as tk
from tkinter import messagebox, simpledialog
import mysql.connector
import logging

# Setup logging
logging.basicConfig(filename='boutique_error.log', level=logging.ERROR)

# Connect to MySQL with error handling
try:
    mycon = mysql.connector.connect(
        host='localhost',
        user='root',
        password='',
        database='sboutique'
    )
    mycur = mycon.cursor()
except mysql.connector.Error as err:
    messagebox.showerror("Database Error", f"Failed to connect: {err}")
    exit()

# Helper functions
def check():
    mycur.execute('SELECT cust_id FROM customer;')
    return [ids[0] for ids in mycur.fetchall()]

def get_bkd_pro(cust_id):
    mycur.execute('SELECT bkd_pro FROM customer WHERE cust_id=%s;', (cust_id,))
    bp = mycur.fetchone()
    return bp[0] if bp else None

# Main Application
class BoutiqueApp:
    def __init__(self, root):
        self.root = root
        self.root.title("S Boutique Management System")
        self.root.geometry("500x400")
        self.root.configure(bg="#f0f8ff")
        self.root.protocol("WM_DELETE_WINDOW", self.on_closing)
        self.create_main_interface()

    def create_main_interface(self):
        tk.Label(self.root, text="Welcome to S Boutique", font=("Helvetica", 16, "bold"), bg="#f0f8ff", fg="#000080").pack(pady=20)
        tk.Button(self.root, text="Customer", width=20, bg="#0092FF", command=self.customer_interface).pack(pady=10)
        tk.Button(self.root, text="Employee", width=20, bg="#00FFBC", command=self.employee_interface).pack(pady=10)
        tk.Button(self.root, text="Employer", width=20, bg="#FFA200", command=self.employer_interface).pack(pady=10)
        tk.Button(self.root, text="Exit", width=20, bg="#93ae55", command=self.on_closing).pack(pady=10)

    def customer_interface(self):
        customer_window = tk.Toplevel(self.root)
        customer_window.title("Customer Interface")
        customer_window.geometry("400x300")
        customer_window.configure(bg="#e6e6fa")
        tk.Button(customer_window, text="Create Account", width=20, bg="#d8bfd8", command=self.create_account).pack(pady=10)
        tk.Button(customer_window, text="Sign In", width=20, bg="#dda0dd", command=self.sign_in).pack(pady=10)

    def create_account(self):
        try:
            custid = simpledialog.askinteger("Input", "Enter your customer ID:")
            if not custid:
                return
            if custid in check():
                messagebox.showerror("Error", "Customer ID already exists.")
                return
            cnam = simpledialog.askstring("Input", "First Name:")
            clnam = simpledialog.askstring("Input", "Last Name:")
            cphno = simpledialog.askstring("Input", "Phone Number:")
            cadrs = simpledialog.askstring("Input", "Address:")
            if not all([cnam, clnam, cphno, cadrs]):
                messagebox.showerror("Error", "All fields are required.")
                return
            qry = 'INSERT INTO customer (cust_id, c_nam, c_lnam, c_phno, c_adrs, bkd_pro) VALUES (%s, %s, %s, %s, %s, NULL);'
            mycur.execute(qry, (custid, cnam, clnam, cphno, cadrs))
            mycon.commit()
            messagebox.showinfo("Success", "Customer account created successfully.")
        except Exception as e:
            logging.error("Create account error", exc_info=True)
            messagebox.showerror("Error", f"An error occurred: {e}")

    def sign_in(self):
        try:
            custid = simpledialog.askinteger("Input", "Enter your customer ID:")
            if not custid or custid not in check():
                messagebox.showerror("Error", "Customer ID does not exist.")
                return
            self.customer_actions(custid)
        except Exception as e:
            logging.error("Sign in error", exc_info=True)
            messagebox.showerror("Error", f"An error occurred: {e}")

    def customer_actions(self, custid):
        actions_window = tk.Toplevel(self.root)
        actions_window.title("Customer Actions")
        actions_window.geometry("400x400")
        actions_window.configure(bg="#ffe4e1")
        tk.Button(actions_window, text="View Bookings", width=25, bg="#ff69b4", command=lambda: self.view_bookings(custid)).pack(pady=10)
        tk.Button(actions_window, text="Book a Product", width=25, bg="#ff1493", command=lambda: self.book_product(custid)).pack(pady=10)
        tk.Button(actions_window, text="Update Details", width=25, bg="#db7093", command=lambda: self.update_details(custid)).pack(pady=10)
        tk.Button(actions_window, text="Cancel Bookings", width=25, bg="#c71585", command=lambda: self.cancel_bookings(custid)).pack(pady=10)

    def view_bookings(self, custid):
        bookings = get_bkd_pro(custid)
        if not bookings:
            messagebox.showinfo("Bookings", "No bookings found.")
        else:
            booked_items = bookings.strip('_').split('_')
            messagebox.showinfo("Bookings", f"Booked Products: {', '.join(booked_items)}")

    def book_product(self, custid):
        try:
            mycur.execute('SELECT pro_id FROM products;')
            product_ids = [prod[0] for prod in mycur.fetchall()]
            pro_id = simpledialog.askstring("Input", "Enter Product ID to book:")
            if pro_id not in product_ids:
                messagebox.showerror("Error", "Product ID does not exist.")
                return
            current_bookings = get_bkd_pro(custid)
            if current_bookings and pro_id in current_bookings.strip('_').split('_'):
                messagebox.showinfo("Info", "Product already booked.")
                return
            new_bookings = (current_bookings or '') + pro_id + '_'
            mycur.execute('UPDATE customer SET bkd_pro=%s WHERE cust_id=%s;', (new_bookings, custid))
            mycon.commit()
            messagebox.showinfo("Success", "Product booked successfully.")
        except Exception as e:
            logging.error("Book product error", exc_info=True)
            messagebox.showerror("Error", f"An error occurred: {e}")

    def update_details(self, custid):
        try:
            mycur.execute('SELECT c_nam, c_lnam, c_phno, c_adrs FROM customer WHERE cust_id=%s;', (custid,))
            details = mycur.fetchone()
            if not details:
                messagebox.showerror("Error", "Customer details not found.")
                return
            cnam = simpledialog.askstring("Input", "First Name:", initialvalue=details[0])
            clnam = simpledialog.askstring("Input", "Last Name:", initialvalue=details[1])
            cphno = simpledialog.askstring("Input", "Phone Number:", initialvalue=details[2])
            cadrs = simpledialog.askstring("Input", "Address:", initialvalue=details[3])
            if not all([cnam, clnam, cphno, cadrs]):
                messagebox.showerror("Error", "All fields are required.")
                return
            mycur.execute('UPDATE customer SET c_nam=%s, c_lnam=%s, c_phno=%s, c_adrs=%s WHERE cust_id=%s;',
                          (cnam, clnam, cphno, cadrs, custid))
            mycon.commit()
            messagebox.showinfo("Success", "Details updated successfully.")
        except Exception as e:
            logging.error("Update details error", exc_info=True)
            messagebox.showerror("Error", f"An error occurred: {e}")

    def cancel_bookings(self, custid):
        try:
            bookings = get_bkd_pro(custid)
            if not bookings:
                messagebox.showinfo("Info", "No bookings to cancel.")
                return
            choice = simpledialog.askstring("Input", "Enter 'A' to cancel all or Product ID to cancel specific booking:")
            if not choice:
                return
            if choice.lower() == 'a':
                mycur.execute('UPDATE customer SET bkd_pro=NULL WHERE cust_id=%s;', (custid,))
                mycon.commit()
                messagebox.showinfo("Success", "All bookings canceled.")
            else:
                booking_list = bookings.strip('_').split('_')
                if choice in booking_list:
                    booking_list.remove(choice)
                    new_bookings = '_'.join(booking_list) + '_' if booking_list else None
                    mycur.execute('UPDATE customer SET bkd_pro=%s WHERE cust_id=%s;', (new_bookings, custid))
                    mycon.commit()
                    messagebox.showinfo("Success", "Booking canceled.")
                else:
                    messagebox.showerror("Error", "Product ID not found in bookings.")
        except Exception as e:
            logging.error("Cancel booking error", exc_info=True)
            messagebox.showerror("Error", f"An error occurred: {e}")

    def employee_interface(self):
        messagebox.showinfo("Coming Soon", "Employee module is under development.")

    def employer_interface(self):
        messagebox.showinfo("Coming Soon", "Employer module is under development.")

    def on_closing(self):
        if mycon.is_connected():
            mycur.close()
            mycon.close()
        self.root.destroy()

if __name__ == "__main__":
    root = tk.Tk()
    app = BoutiqueApp(root)
    root.mainloop()
